# Quantile regression of QTL association testing
This notebook implements a workflow for using [quantile regression of QTL](https://pubmed.ncbi.nlm.nih.gov/37333162/) to perform quantile QTL association testing. 

## Input

- molecular phenotype files: the whole `bed.gz` file containing the table for the molecular phenotype. It should have a companion index file in `tbi` format.

    The header of the bed.gz is per the [TensorQTL](https://github.com/broadinstitute/tensorqtl) convention:

    >    Phenotypes must be provided in BED format, with a single header line starting with # and the first four columns corresponding to: chr, start, end, phenotype_id, with the remaining columns corresponding to samples (the identifiers must match those in the genotype input). The BED file should specify the center of the cis-window (usually the TSS), with start == end-1.


- List of genotypes in PLINK binary format (`bed`/`bim`/`fam`) for each gene, previously processed through our genotype QC pipelines.
- Covariate file, a file with #id + samples name as colnames and each row a covariate: fixed and known covariates as well as hidden covariates recovered from factor analysis.
- Optionally, a list of traits (genes, regions of molecular features etc) to analyze. In this case, AD gene list and tad list are used, and only analyze the overlapped genes of phenotype data and 76 genes from the AD GWAS list which have AD signal. The tad list is used to perform genomic region analysis and get customized window size.

    

## Output

For each gene, several of summary statistics files are generated, including cis nominal test statistics for each test.

The columns of cis nominal association result are as follows:

- pheno_id: Molecular trait identifier.(gene)
- geno_id: ID of the variant (rsid or chr:position:ref:alt)
- chr : Variant chromosome.
- pos : Variant chromosomal position (basepairs).
- ref : Variant reference allele (A, C, T, or G).
- alt : Variant alternate allele.
- heterogeneity： heterogeneity index of each snp
- is_alt_minor: Whether the minor allele frequency is for the alternate allele
- maf: Minor allele frequency
- ma_samples: Number of samples carrying the minor allele
- ma_count: Total number of minor alleles across individuals
- p_qr_cauchy(composite-p value using cauchy combination method): the integrated QR p-value across multiple quantile levels. 
- p_qr(default composite-p value in QRank package): the integrated QR p-value across multiple quantile levels.    
- p_qr_0.1 to p_qr_0.9: quantile-specific QR p-values for the quantile levels 0.1, 0.2, ..., 0.9.   
- coef_qr_0.1 to coef_qr_0.9: quantile-specific QR coefficients for the quantile levels 0.1, 0.2, ..., 0.9.   

# Command interface 

In [3]:
!sos run /home/al4225/project/xqtl-pipeline/code/quantile_qtl/quantile_qtl.ipynb -h

usage: sos run /home/al4225/project/xqtl-pipeline/code/quantile_qtl/quantile_qtl.ipynb
               [workflow_name | -t targets] [options] [workflow_options]
  workflow_name:        Single or combined workflows defined in this script
  targets:              One or more targets to generate
  options:              Single-hyphen sos parameters (see "sos run -h" for details)
  workflow_options:     Double-hyphen workflow-specific parameters

Workflows:
  get_analysis_regions
  qrqtl

Global Workflow Options:
  --genoFile VAL (as path, required)
                        A list of file paths for genotype data.
  --phenoFile  paths

                        One or multiple lists of file paths for phenotype data.
  --covFile  paths

                        Covariate file path
  --region-list . (as path)
                        Optional: if a region list is provide the analysis will
                        be focused on provided region. The LAST column of this
                        list will 

nohup sos run /home/al4225/project/quantile_qtl/rq_pqtl_ad_cis.ipynb qrqtl \
    --name snuc_pseudo_bulk_mic \
    --genoFile /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/plink_by_gene/extended_cis_before_winsorize_plink_files/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.plink_files_list.txt \
    --phenoFile /home/al4225/pseudobulk/output/data_preprocessing/phenotype_data/mic/pheno_list.txt \
    --region-list /home/al4225/project/quantile_qtl/ad_geneid_list.txt \
    --covFile /mnt/vast/hpc/homes/al4225/pseudobulk/output/data_preprocessing/covariate_data/Mic.log2cpm.mic.rosmap_cov.ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.snuc_pseudo_bulk.related.plink_qc.extracted.pca.projected.Marchenko_PC.gz \
    --cwd /home/al4225/project/quantile_qtl/output/mic3/ \
    --container /home/al4225/project/quantile_container/quantqtl.sif \
    --mem 200G -J 50 -c /home/al4225/project/quantile_qtl/csg.yml -q csg -s build >./nohup_quantile_mic.out6 &
    

## Global parameter settings

In [ ]:
[global]
# A list of file paths for genotype data. 
parameter: genoFile = path
# One or multiple lists of file paths for phenotype data.
parameter: phenoFile = paths
# Covariate file path
parameter: covFile = paths
# Optional: if a region list is provide the analysis will be focused on provided region. 
# The LAST column of this list will contain the ID of regions to focus on
# Otherwise, all regions with both genotype and phenotype files will be analyzed
parameter: region_list = path()
# Optional: if a region name is provided 
# the analysis would be focused on the union of provides region list and region names
parameter: region_name = []
parameter: cwd = path("output")
# It is required to input the name of the analysis
parameter: name = str
# path to utility script. In the future we will consolidate this into an R package.
parameter: utils_R = path("pipeline/xqtl_utils.R")
parameter: container = ""
import re
parameter: entrypoint= ('micromamba run -a "" -n' + ' ' + re.sub(r'(_apptainer:latest|_docker:latest|\.sif)$', '', container.split('/')[-1])) if container else ""
# For cluster jobs, number commands to run per job
parameter: job_size = 5
# Wall clock time expected
parameter: walltime = "20h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 2
# Name of phenotypes
parameter: phenotype_names = [f'{x:bn}' for x in phenoFile]
utils_R = f"{utils_R:a}"

In [1]:
# this aims to show an example of pheno list
import pandas as pd
pheno = pd.read_csv('/home/al4225/pseudobulk/output/data_preprocessing/phenotype_data/mic/pheno_list.txt', sep = '\t')
print(pheno.head())
rows, cols = pheno.shape
print(f"there are {rows} rows(genes) and {cols} cols in pheno list file")


    #chr    start      end               ID  \
0  chr12   389319   389320  ENSG00000073614   
1  chr12   752578   752579  ENSG00000060237   
2  chr12   990052   990053  ENSG00000002016   
3  chr12   990508   990509  ENSG00000082805   
4  chr12  1688573  1688574  ENSG00000006831   

                                                path  
0  /home/al4225/pseudobulk/output/data_preprocess...  
1  /home/al4225/pseudobulk/output/data_preprocess...  
2  /home/al4225/pseudobulk/output/data_preprocess...  
3  /home/al4225/pseudobulk/output/data_preprocess...  
4  /home/al4225/pseudobulk/output/data_preprocess...  
there are 7762 rows(genes) and 5 cols in pheno list file


In [2]:
# this aims to show an example of resion list: last col: gene_id
import pandas as pd
ad_id_list = pd.read_csv('/home/al4225/project/quantile_qtl/ad_geneid_list.txt', sep = '\t')
print(ad_id_list.head())
rows, cols = ad_id_list.shape
print(f"there are {rows} rows(genes) and {cols} cols in ad_id_list file")


   #chr      start        end          gene_id
0  chr1  109397917  109397918  ENSG00000134243
1  chr1  207496146  207496147  ENSG00000203710
2  chr2    9556731    9556732  ENSG00000151694
3  chr2   37324832   37324833  ENSG00000115825
4  chr2  105744911  105744912  ENSG00000071051
there are 76 rows(genes) and 4 cols in ad_id_list file


In [ ]:
# this aims to show an example of geno list.
import pandas as pd
geno = pd.read_csv('/mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/plink_by_gene/extended_cis_before_winsorize_plink_files/ROSMAP_NIA_WGS.leftnorm.bcftools_qc.plink_qc.plink_files_list.txt', sep = '\t')

print(geno.head())
rows, cols = geno.shape
print(f"there are {rows} rows(genes) and {cols} cols in geno list file")


               #id                                                dir
0  ENSG00000008128  /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/...
1  ENSG00000008130  /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/...
2  ENSG00000067606  /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/...
3  ENSG00000069424  /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/...
4  ENSG00000078369  /mnt/vast/hpc/csg/FunGen_xQTL/ROSMAP/Genotype/...
there are 26806 rows(genes) and 2 cols in geno list file


## Quantile regression to call xqlt

In [ ]:
[get_analysis_regions: shared = "regional_data"]
# input is genoFile, phenoFile, covFile and optionally region_list. If region_list presents then we only analyze what's contained in the list.
# regional_data should be a dictionary with:
# 1. a list of tuples: {data: [(gene_1.genotype, condition_1, cov_1), (gene_2.genotype, condition_1, cov_1, condition_2, cov_2), ...]} each element may not be of the same length
# 2. a list of region meta_info: {meta_info: ( "chr:start-end",gene_1,"cond_1"), ("chr:start-end",gene_2, "cond_1','cond_2"), ...]}
import pandas as pd
import os
genoFile = pd.read_csv(genoFile, sep = "\t", header=0)

if len(phenoFile) != len(covFile):
    raise ValueError("Number of input phenotypes files must match that of covariates files")
if len(phenoFile) != len(phenotype_names):
    raise ValueError("Number of input phenotypes files must match the number of phenotype names")
## pos and covar are condition specific, this way when there is no phenotype file, there is na in the corresponding column.
phenoFile = [pd.read_csv(x, sep = "\t", header=0).assign(pos = lambda y:y['#chr']+':'+y['start'].astype("str")+'-'+
                                              y['end'].astype("str")).assign(cov_path = z, cond = a ).drop(columns = ["#chr","start","end"]).rename(columns = {"ID":"#id"})   
             for x,z,a in zip(phenoFile,covFile,phenotype_names)]
for i in range(len(phenoFile)):
    genoFile = genoFile.merge(phenoFile[i], on='#id', how='left', suffixes = (f'{i}_x', f'{i}_y'))

# remove id that has no phenotype.

genoFile = genoFile[~genoFile.drop(columns=['#id',"dir"]).isna().all(axis=1)]    

if len(genoFile.index) == 0:
    raise ValueError("No region overlap between genotype #id and any of the phenotypes ID")

# Get position for meta_data
pos_col = [col for col in genoFile.columns if col.startswith('pos')]
genoFile.index = pd.Series(genoFile[pos_col].values.flatten()).dropna()
# Get the conditions strings for each ID
cond_col = [col for col in genoFile.columns if col.startswith('cond')]
genoFile["phenotype_names"] = ["','".join(pd.Series((x)).dropna()) for x in genoFile[cond_col].to_dict("split")["data"]]
# Clean up
genoFile = genoFile.drop(columns = cond_col).drop(columns = pos_col)


region_ids = []

# If region_list is provided, read the file and extract IDs
if region_list.is_file():
    region_list_df = pd.read_csv(region_list, sep = "\t", header=None, comment = "#")
    region_ids = region_list_df.iloc[:, -1].unique()  # Extracting the last column for IDs

# If region_name is provided, include those IDs as well
# --region-name A B C will result in a list of ["A", "B", "C"] here
if len(region_name) > 0:
    region_ids = list(set(region_ids).union(set(region_name)))

# If either region_list or region_name is provided, filter the genoFile
if len(region_ids) > 0:
    genoFile = genoFile[genoFile['#id'].isin(region_ids)]

file_inv = genoFile.drop(columns = ["#id", "phenotype_names"]).to_dict("split")
file_inv['data'] = [[value for value in sublist if not pd.isna(value)] for sublist in file_inv['data']] 


## There will alwayse be genotype file due to left join,
## There will alwayse be covar file as len(covFile) must == len(PhenoFile), and covar column is the same string accross all rows
## So only if there is no bed.gz there will be problem.
regional_data = {"data":file_inv["data"],"meta_info": genoFile[["#id","phenotype_names"]].reset_index().to_dict("split")['data'] }

# Recreate file_inv based on the filtered genoFile
file_inv = genoFile.drop(columns=["#id", "phenotype_names"]).to_dict("split")
file_inv['data'] = [[value for value in sublist if not pd.isna(value)] for sublist in file_inv['data']] 

# Recreate the regional_data based on the filtered data
regional_data = {"data": file_inv["data"],
                 "meta_info": genoFile[["#id", "phenotype_names"]].reset_index().to_dict("split")['data']}
print("regional_data first 2 rows:")
for row in regional_data['data'][:2]:
    print(row)
for row in regional_data['meta_info'][:2]:
    print(row)
num_of_rows = len(regional_data['meta_info'])
print(f"meta_info has {num_of_rows} rows.")

In [ ]:
[qrqtl_1]
parameter: imiss = 1
# Minor allele count cutoff
parameter: mac = 0
import pandas as pd
N = len(pd.read_csv(covFile, sep = "\t",nrows = 1).columns) - 1 # Use the header of covariate file for it being the intersect of geno/pheno/cov.
# Minor allele frequency cutoff. It will overwrite minor allele cutoff.
parameter: maf = mac/(2.0*N)

depends: sos_variable("regional_data")

def group_by_region(lst, data):
    vector = [len(x) for x in data]
    return [lst[sum(vector[:i]):sum(vector[:i+1])] for i in range(len(vector))]

meta_info = regional_data['meta_info']
input: regional_data["data"], group_by = lambda x: group_by_region(x, regional_data["data"]), group_with = "meta_info"
output: qr_nominal_result = f'{cwd:a}/{step_name[:-2]}/{name}.{_meta_info[1]}.qr_nominal_result.rds'
task: trunk_workers = 1, trunk_size = job_size, walltime = walltime, mem = mem, cores = numThreads, tags = f'{step_name}_{_output[0]:bn}'
R: expand = '${ }', stdout = f"{_output:n}.stdout", stderr = f"{_output:n}.stderr", container = container, entrypoint = entrypoint, input = utils_R
library("tidyverse")
library(QRank)
library(quantreg)

    # Load regional association data
    fdat = load_regional_quantile_data(genotype = ${_input[0]:anr},
                                          phenotype = c(${",".join(['"%s"' % x.absolute() for x in _input[1::2]])}),
                                          covariate = c(${",".join(['"%s"' % x.absolute() for x in _input[2::2]])}),
                                          region = "${_meta_info[0]}",
                                          conditions = c('${_meta_info[2]}'),
                                          maf_cutoff = ${maf},
                                          mac_cutoff = ${mac},
                                          imiss_cutoff = ${imiss},
                                          y_as_matrix = FALSE)
    # Print the head of Y, X_data, and covar
    print("Head of Y:")
    print(dim(fdat$Y[[1]]))
    print(head(fdat$Y[[1]]))

    print("Head of X_data:")
    print(dim(fdat$X_data[[1]]))

    print("Head of covar:")
    print(dim(fdat$covar[[1]]))
    print(head(fdat$covar[[1]]))
    
    # QRank test
    ## Cauchy combination
    cauchy.meta <- function(pvals) {
    # Check input
    pvals = pvals[!is.na(pvals)]
    if (length(pvals) == 0) {
        return(NA)
    }
    # pvals[pvals == 0] = 2.2E-308
    # Convert to Cauchy
    cauchy = 1 / (pvals * pi)
    cauchy[pvals >= 1e-15] = tanpi(0.5 - pvals[pvals >= 1e-15])
    stats = mean(cauchy)
    p = pcauchy(q = stats, lower.tail = F)
    return(p)
    }
    

start_time <- proc.time()
# Qrank for p 
#fitted = list()
fitted <- vector("list", length(fdat$Y))
cat("length(fdat$Y)", length(fdat$Y), "\n")
for (r in 1:length(fdat$Y)) {
    st = proc.time()
    qntl <- (1:9) / 10
    genotype_df = fdat$X_data[[r]]
    phenotype_df = fdat$Y[[r]]
    colnames(phenotype_df) = c('${_meta_info[1]}')
    covariates_df = fdat$covar[[r]]

    df.p <- data.frame()
    additional_info_df <- data.frame(pheno_id = character(), geno_id = character(), is_alt_minor = logical(), maf = numeric(), ma_samples = integer(), ma_count = integer())

    for (i in 1:ncol(genotype_df)) {
        geno_data <- genotype_df[, i]
        alt_allele_freq <- mean(geno_data) / 2
        if (alt_allele_freq > 0.5) {
            MAF <- 1 - alt_allele_freq
            is_minor <- FALSE
        } else {
            MAF <- alt_allele_freq
            is_minor <- TRUE
        }
        af <- MAF
        if (is_minor) {
            ma_samples <- sum(geno_data > 0.5)
            ma_count <- round(sum(geno_data))
        } else {
            ma_samples <- sum(geno_data < 1.5)
            ma_count <- round(2 * length(geno_data) - sum(geno_data))
        }
        ma_count <- as.integer(ma_count)
        additional_info_row <- data.frame(pheno_id = colnames(phenotype_df), geno_id = colnames(genotype_df)[i], is_alt_minor = is_minor, maf = af, ma_samples = ma_samples, ma_count = ma_count)
        additional_info_df <- rbind(additional_info_df, additional_info_row)

        p.qr <- QRank(gene = as.matrix(phenotype_df), 
                          cov = as.matrix(covariates_df), 
                          snp = geno_data, 
                          tau = qntl)
        heterogeneity_result <- heter.QRank(p.qr)
        heterogeneity_metric <- heterogeneity_result$`heter.index`
        col_names <- c('p_qr_cauchy', 'p_qr', paste0('p_qr_', qntl), 'heterogeneity')
        df.row <- data.frame(pheno_id = colnames(phenotype_df), geno_id = colnames(genotype_df)[i], 'p_qr_cauchy' = cauchy.meta(as.numeric(p.qr$quantile.specific.pvalue)),'p_qr' = p.qr$composite.pvalue ,t(p.qr$quantile.specific.pvalue), heterogeneity = heterogeneity_metric)
        names(df.row) <- c("pheno_id", "geno_id", col_names)
        df.p <- rbind(df.p, df.row)
    }
    cis_nominal_p <- merge(df.p, additional_info_df)
    cat("qrank loaded ", dim(cis_nominal_p), "\n")
    print(head(cis_nominal_p))

start_time2 <- proc.time()
# quantile regression--get beta
# coef result of tau from 0.1 to 0.9
pheno.mat = as.matrix(phenotype_df)
geno.mat = as.matrix(genotype_df)
cova.mat = as.matrix(covariates_df)
result_table_br <- data.frame(pheno_id = character(),
                           geno_id = character(),
                           tau = numeric(),
                           predictor_coef = numeric(),
                           stringsAsFactors = FALSE)

for (tau in seq(0.1, 0.9, by = 0.1)) {
     for (n in 1:ncol(geno.mat)){
      
      response <- pheno.mat
      predictor <- geno.mat[, n]
      if (!is.null(cova.mat)) {
        X <- cbind(1, predictor, cova.mat)
      } else {
        X <- cbind(1, predictor)
      }
      
      pheno_id <- colnames(pheno.mat)
      geno_id <- colnames(geno.mat)[n]

      # Using rq.fit.br
      mod_br <- rq.fit.br(X, response, tau = tau)
      predictor_coef_br <- mod_br$coefficients[2] #get the coefficient of variant
      
      row_br <- data.frame(pheno_id = pheno_id, geno_id = geno_id, tau = tau, predictor_coef = predictor_coef_br, stringsAsFactors = FALSE)
      result_table_br <- rbind(result_table_br, row_br)
    }
}

result_table_wide = result_table_br %>% pivot_wider(names_from = tau, values_from = predictor_coef)
colnames(result_table_wide)[3:ncol(result_table_wide)] <- paste0("coef_qr_", colnames(result_table_wide)[3:ncol(result_table_wide)])
 cat("rq.fit.br result loaded ", dim(result_table_wide), "\n")
print(head(result_table_wide))
end_time2 <- proc.time()
cat("rq loaded and time is", end_time2 - start_time2, "seconds\n")

# combined p(composite-p from default QRank and cauchy method) and coef table
## QR summary statistics
## row: variant
## column: p-value, coefficient and other parameters
qr_nominal_result = merge(cis_nominal_p, result_table_wide, by = c("pheno_id", "geno_id")) %>%
  separate(geno_id, into = c("chr", "pos", "ref", "alt"), sep = "[:_]", remove = FALSE) %>%
  mutate(chr = as.numeric(gsub("chr", "", chr)), pos = as.numeric(pos))
  cat("combined p and beta result loaded: ",dim(qr_nominal_result) , "\n")
column_names <- colnames(qr_nominal_result)

#change order
new_order <- c(column_names[3:6], column_names[1:2], column_names[7:17], column_names[23:length(column_names)])
qr_nominal_result <- qr_nominal_result[new_order]
print(head(qr_nominal_result))

fitted[[r]]$analysis_time <- proc.time() - st
fitted[[r]]$df.p = df.p
fitted[[r]]$additional_info_df = additional_info_df
fitted[[r]]$p.qr = p.qr
fitted[[r]]$cis_nominal_p = cis_nominal_p
fitted[[r]]$result_table_wide = result_table_wide
fitted[[r]]$qr_nominal_result = qr_nominal_result

}
names(fitted) <- names(fdat$Y)
saveRDS(fitted, ${_output:ar})   

In [ ]:
[qrqtl_2]
input: group_by = "all"
output: f'{cwd}/{name}.quantqtl_output2.txt'
python: expand= "$[ ]", stderr = f'{_output:n}.stderr', stdout = f'{_output:n}.stdout'
    import pandas as pd
    pd.DataFrame({"output" : [$[_input:ar,]]}).to_csv("$[_output]",index = False ,header = False, sep = "\t")